# Conditional LSTM

In [1]:
import os
import numpy as np

from pybtex.database import parse_file

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.ldamodel import LdaModel
from gensim.models.lsimodel import LsiModel
from gensim.corpora.dictionary import Dictionary

D:\giuse\Conda\envs\thesis\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
bib_data = parse_file('data/anthology+abstracts.bib')

In [ ]:
list(bib_data.entries.keys())[-1]

In [ ]:
len(list(bib_data.entries.keys()))

In [ ]:
bib_data.entries['lieberman-etal-1965-automatic'].fields['year']

In [ ]:
for k in bib_data.entries.keys():
    try:
        year = bib_data.entries[k].fields['year']
        abstract = bib_data.entries[k].fields['abstract']
        
        if year > '2015':
            f = open('data/datasets/abstracts_%s.txt' %year, 'a')
            f.write(abstract)
            f.close()
            
    except (KeyError, UnicodeEncodeError): # entries without abstracts are excluded
        pass

In [ ]:
# eliminate stop words
def tokenize_input(input):
    # make everything lowercase
    input = input.lower()

    # use tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # filter out stopwords
    final = filter(lambda token: token not in stopwords.words('english'), tokens)
    
    # end result in final
    return " ".join(final)

In [ ]:
for year in range(2016,2022):        
        with open('data/datasets/abstracts_%s.txt' %year) as abstr:
            lines = abstr.readlines()
            processed = tokenize_input(lines[0])
            
            # create individual year files
            y = open('data/datasets/%s.txt' %year, 'a')
            y.write(processed)
            
            # create all years file
            a = open('data/datasets/all.txt', 'a')
            a.write(processed)
            
            y.close()
            a.close()

In [ ]:
for k in bib_data.entries.keys():
    try:
        year = bib_data.entries[k].fields['year']
        abstract = bib_data.entries[k].fields['abstract']
        
        if year > '2015':
            f = open('data/datasets/abstracts_%s.txt' %year, 'a')
            f.write(abstract + '\n')
            f.close()
            
    except (KeyError, UnicodeEncodeError): # entries without abstracts are excluded
        pass

In [392]:
f = open('data/datasets/abstracts_2020.txt')
text = f.read()
abstracts = text.split('\n')

In [393]:
trimmed = [remove_stopwords(a) for a in abstracts]
lowercase = [a.lower() for a in trimmed]

In [394]:
tokenizer = RegexpTokenizer(r'\w+')
tokenized = [tokenizer.tokenize(a) for a in lowercase]

In [395]:
# total number of tokenized abstracts
len(tokenized)

6499

In [396]:
# example of tokenized abstract
tokenized[2]

['we',
 'introduce',
 'smartcitecon',
 'scc',
 'java',
 'api',
 'extracting',
 'explicit',
 'implicit',
 'citation',
 'context',
 'academic',
 'literature',
 'english',
 'the',
 'tool',
 'built',
 'support',
 'vector',
 'machine',
 'svm',
 'model',
 'trained',
 'set',
 '7',
 '058',
 'manually',
 'annotated',
 'citation',
 'context',
 'sentences',
 'curated',
 '34',
 '000',
 'papers',
 'acl',
 'anthology',
 'the',
 'model',
 '19',
 'features',
 'achieves',
 'f1',
 '85',
 '6',
 'scc',
 'supports',
 'pdf',
 'xml',
 'json',
 'files',
 'out',
 'of',
 'box',
 'provided',
 'conformed',
 'certain',
 'schemas',
 'the',
 'api',
 'supports',
 'single',
 'document',
 'processing',
 'batch',
 'processing',
 'parallel',
 'it',
 'takes',
 '12',
 '45',
 'seconds',
 'average',
 'depending',
 'format',
 'process',
 'document',
 'dedicated',
 'server',
 '6',
 'multithreaded',
 'cores',
 'using',
 'scc',
 'extracted',
 '11',
 '8',
 'million',
 'citation',
 'context',
 'sentences',
 'textasciitilde',
 '33'

In [397]:
# single word
tokenized[0][4]

'patents'

In [398]:
words = [word for abstract in tokenized for word in abstract] # could use itertools to improve performance

In [399]:
len(words)

606420

### Dictionary

In [475]:
dct = Dictionary(tokenized)
dct[42]

'little'

In [401]:
len(dct)

24741

In [476]:
# trim dict at 1000 tokens
dct.filter_extremes(no_below=1, no_above=0.1, keep_n=928)

In [477]:
len(dct)

928

In [478]:
dct[42]

'entities'

In [410]:
print(dct)

Dictionary(811 unique tokens: ['0', '20', 'achieves', 'advantage', 'architecture']...)


In [589]:
corpus = [dct.doc2bow(text) for text in tokenized]

### LDA (unordered)

In [ ]:
lda = LdaModel(corpus, num_topics=10, id2word=dct)

In [ ]:
lda.show_topics()

### Doc2Vec (unordered)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokenized)]

In [ ]:
doc2vec_model = Doc2Vec(documents, vector_size=10, window=2, min_count=1, workers=4)

### LSA (ordered)

In [ ]:
lsi = LsiModel(corpus, id2word=dct, num_topics=10, decay=0.5)

In [ ]:
lsi.show_topics()

In [ ]:
lsi.show_topic(8, topn=20)

In [ ]:
lsi[corpus]

In [ ]:
topic_representation = lsi.projection.u

In [ ]:
lsi.projection.s.shape

In [ ]:
topic_representation.shape

## Conditioned LSTM with Word2Vec embeddings #1

### Dataset

In [551]:
import torch
import pandas as pd
from collections import Counter

class Dataset(torch.utils.data.Dataset):
    def __init__(self, words, sequence_length=5): # TODO: incorporate dictionary
        self.words = words[:2000]
        self.uniq_words = self.get_uniq_words()
        self.sequence_length = sequence_length

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]),
        )

### Embeddings

In [592]:
f = open('data/datasets/abstracts_2020.txt')
text = f.read()
abstracts = text.split('\n')

In [599]:
words = [word for abstract in tokenized for word in abstract] # could use itertools to improve performance

In [600]:
len(words)

606420

In [553]:
import gensim

w2v_model = gensim.models.Word2Vec(abstracts, min_count=1, vector_size=256, epochs=5)

In [554]:
vectors = w2v_model.wv.vectors[:928]

In [555]:
vectors.shape

(928, 256)

In [556]:
weights = torch.FloatTensor(vectors)

### Model architecture

In [557]:
from torch import nn

class Model(nn.Module):
    def __init__(self, dataset, lstm_size=256, emdedding_dim=256, num_layers=2, dropout=0.2):
        super(Model, self).__init__()
        self.lstm_size = lstm_size
        self.embedding_dim = emdedding_dim
        self.num_layers = num_layers
        self.sequence_length = dataset.sequence_length
        self.dropout = dropout

        n_vocab = len(dataset.uniq_words)
        
        self.embedding = nn.Embedding.from_pretrained(weights)
        
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=self.dropout,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)

        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, self.sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, self.sequence_length, self.lstm_size))

### Training

In [558]:
from torch import nn, optim
from torch.utils.data import DataLoader

def train(dataset, model, batch_size=128, max_epochs=3):
    model.train()

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
    )

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.02)

    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(model.sequence_length)

        for batch, (x, y) in enumerate(dataloader):

            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()
            
            if batch % 50 == 0:
                print({ 'Epoch': epoch, 'Batch': batch, 'Loss': loss.item() })

### Generation

In [576]:
from itertools import cycle, islice

def generate(dataset, model, text, next_words=100):
    output = text.split(' ')
    model.eval()

    state_h, state_c = model.init_state(model.sequence_length)

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in output[i:]]])        
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        output.append(dataset.index_to_word[word_index])

        return ' '.join(output)

In [577]:
input_text = 'in this paper we propose'

dataset = Dataset(words, sequence_length=5)
model = Model(dataset)

train(dataset, model, max_epochs=1)

{'Epoch': 0, 'Batch': 0, 'Loss': 6.8361334800720215}


In [579]:
generate(dataset, model, text=text, next_words=20)

In [580]:
dataset.index_to_word[62]

'documents'

In [449]:
words = words[:2000]

In [472]:
n_vocab = len(dataset.uniq_words)
dataset.uniq_words[810]

'evaluates'

In [590]:
len(dataset.uniq_words)

928

## Conditioned LSTM with Word2Vec embeddings #2

In [724]:
import re
import random
import pickle

import torch.nn.functional as F

In [946]:
# read file
f = open('data/datasets/abstracts_2021.txt')
text = f.read()
abstracts = text.split('\n')

# count of movie plot summaries
len(abstracts)

1735

In [927]:
# read pickle file
pickle_in = open("plots_text.pickle","rb")
abstracts = pickle.load(pickle_in)

# count of movie plot summaries
len(abstracts)

500

In [947]:
abstracts = [re.sub("[^a-z', ]", "", a) for a in abstracts]

In [948]:
random.sample(abstracts, 2)

['he aim of the paper is twofold  to automatically predict the ratings assigned by viewers to  categories available for  talks in a multilabel classification task and  to determine what types of features drive classification accuracy for each of the categories he focus is on features of language usage from five groups pertaining to syntactic complexity, lexical richness, registerbased ngram measures, informationtheoretic measures and style measures e show that a ecurrent eural etwork classifier trained exclusively on withintext distributions of such features can reach relatively high levels of overall accuracy  across the  categories e find that features from two groups are strong predictors of the affective ratings across all categories and that there are distinct patterns of language usage for each rating category',
 'his paper addresses the problem of sentiment analysis for opara, a codeswitching language between uarani and panish e first collect a corpus of uaranidominant tweets an

In [949]:
# create sequences of default length 5 tokens
def create_seq(text, seq_len = 5):
    
    sequences = []

    # if the number of tokens in 'text' is greater than 5
    if len(text.split()) > seq_len:
      for i in range(seq_len, len(text.split())):
        # select sequence of tokens
        seq = text.split()[i-seq_len:i+1]
        # add to the list
        sequences.append(" ".join(seq))

      return sequences

    # if the number of tokens in 'text' is less than or equal to 5
    else:
      
      return [text]

In [950]:
seqs = [create_seq(a) for a in abstracts]

# merge list-of-lists into a single list
seqs = sum(seqs, []) # could use itertools to improve performance

# count of sequences
len(seqs)

216543

In [951]:
# create inputs and targets (x and y)
x = []
y = []

for s in seqs:
  x.append(" ".join(s.split()[:-1]))
  y.append(" ".join(s.split()[1:]))

In [952]:
type(x)

list

In [953]:
# create integer-to-token mapping
int2token = {}
counter = 0

for w in set(" ".join(abstracts).split()):
  int2token[counter] = w
  counter += 1

# create token-to-integer mapping
token2int = {t: a for a, t in int2token.items()}

token2int["the"], int2token[4299]

(2366, 'tracks,')

In [954]:
vocab_size = len(int2token)
vocab_size

15179

In [965]:
def get_integer_seq(seq):
  return [token2int[w] for w in seq.split()]

# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x]
y_int = [get_integer_seq(i) for i in y]

In [966]:
len(x_int),len(y_int)

(216543, 216543)

In [967]:
# This may cause problems down the line (check indices)

x_int = list(filter(lambda x: (len(x) == 5), x_int))
y_int = list(filter(lambda y: (len(y) == 5), y_int))

In [968]:
len(x_int),len(y_int)

(216541, 216541)

In [969]:
# convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

In [980]:
def get_batches(arr_x, arr_y, batch_size):
         
    # iterate through the arrays
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
      x = arr_x[prv:n]
      y = arr_y[prv:n]
      prv = n
      yield x, y

In [985]:
class Model(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=2, drop_prob=0.2, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, 256)

        ## define the LSTM
        self.lstm = nn.LSTM(256, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        x = x.long()

        ## pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(lstm_output)
        
        #out = out.contiguous().view(-1, self.n_hidden) 
        out = out.reshape(-1, self.n_hidden) 

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available
        if (torch.cuda.is_available()):
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        # if GPU is not available
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [986]:
# instantiate the model
model = Model()

model.cpu()

print(model)

Model(
  (emb_layer): Embedding(15179, 256)
  (lstm): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=15179, bias=True)
)


In [994]:
def train(model, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):
    
    # optimizer
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    
    # loss
    criterion = nn.CrossEntropyLoss()
    
    # push model to CPU
    model.cpu()
    
    counter = 0

    model.train()

    for e in range(epochs):

        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        for x, y in get_batches(x_int, y_int, batch_size):
            counter+= 1
            
            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            # push tensors to GPU
            inputs, targets = inputs.cpu(), targets.cpu()

            # detach hidden states
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            model.zero_grad()
            
            # get the output from the model
            output, h = model(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1).long())

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)

            # update weigths
            opt.step()            
            
            if counter % print_every == 0:
            
              print("Epoch: {}/{} -".format(e+1, epochs),
                    "Step: {} -".format(counter),
                    "Loss: {}".format(loss))

In [ ]:
train(model, batch_size=8, epochs=1, print_every=10)

In [707]:
# predict next token
def predict(model, tkn, h=None):
         
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to CPU
  inputs = inputs.cpu()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = model(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], h

In [708]:
# function to generate text
def sample(model, size, prime='it is'):
        
    # push to CPU
    model.cpu()
    
    model.eval()

    # batch size is 1
    h = model.init_hidden(1)

    toks = prime.split()

    # predict next token
    for t in prime.split():
      token, h = predict(model, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(model, toks[-1], h)
        toks.append(token)

    return ' '.join(toks)

In [729]:
sample(model, 10, 'here comes')

'here comes on his time and a good train and a good'